In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import optimizers
import keras_cv
from keras_cv import bounding_box
import os
from luketils import visualization

2023-02-02 14:09:18.318562: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


You do not have Waymo Open Dataset installed, so KerasCV Waymo metrics are not available.


In [2]:
BATCH_SIZE = 16
cEPOCHS = int(os.getenv("EPOCHS", "1"))
CHECKPOINT_PATH = os.getenv("CHECKPOINT_PATH", "checkpoint/")
INFERENCE_CHECKPOINT_PATH = os.getenv("INFERENCE_CHECKPOINT_PATH", CHECKPOINT_PATH)

In [3]:
train_ds, ds_info = keras_cv.datasets.pascal_voc.load(
    split='train', bounding_box_format='xywh', batch_size=8
)

2023-02-02 14:09:28.048962: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

SSLError: HTTPSConnectionPool(host='pjreddie.com', port=443): Max retries exceeded with url: /media/files/VOCtest_06-Nov-2007.tar (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1131)')))

In [ ]:
dataset, dataset_info = keras_cv.datasets.pascal_voc.load(
    split="train", bounding_box_format="xywh", batch_size=BATCH_SIZE
)

class_ids = [
    "Aeroplane",
    "Bicycle",
    "Bird",
    "Boat",
    "Bottle",
    "Bus",
    "Car",
    "Cat",
    "Chair",
    "Cow",
    "Dining Table",
    "Dog",
    "Horse",
    "Motorbike",
    "Person",
    "Potted Plant",
    "Sheep",
    "Sofa",
    "Train",
    "Tvmonitor",
    "Total",
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

In [ ]:
def visualize_dataset(dataset, bounding_box_format):
    example = next(iter(dataset))
    images, boxes = example["images"], example["bounding_boxes"]
    visualization.plot_bounding_box_gallery(
        images,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        y_true=boxes,
        scale=4,
        rows=3,
        cols=3,
        show=True,
        thickness=4,
        font_scale=1,
        class_mapping=class_mapping,
    )

In [ ]:
visualize_dataset(dataset, bounding_box_format="xywh")

In [ ]:
train_ds, train_dataset_info = keras_cv.datasets.pascal_voc.load(
    bounding_box_format="xywh", split="train", batch_size=BATCH_SIZE
)
val_ds, val_dataset_info = keras_cv.datasets.pascal_voc.load(
    bounding_box_format="xywh", split="validation", batch_size=BATCH_SIZE
)

In [ ]:
random_flip = keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xywh")
rand_augment = keras_cv.layers.RandAugment(
    value_range=(0, 255),
    augmentations_per_image=2,
    # we disable geometric augmentations for object detection tasks
    geometric=False,
)

In [ ]:
def augment(inputs):
    # In future KerasCV releases, RandAugment will support
    # bounding box detection
    inputs["images"] = rand_augment(inputs["images"])
    inputs = random_flip(inputs)
    return inputs

In [ ]:
train_ds = train_ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
visualize_dataset(train_ds, bounding_box_format="xywh")

In [ ]:
def dict_to_tuple(inputs):
    return inputs["images"], inputs["bounding_boxes"]

In [ ]:
train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
model = keras_cv.models.RetinaNet(
    # 감지할 클래스 의 갯수
    classes=20,
    # 지원하는 bounding box formats에 대해서 더 자세히 표기 위해 아래 링크를 참조합니다.
    # https://keras.io/api/keras_cv/bounding_box/
    bounding_box_format="xywh",
    # KerasCV는 사전 학습한 모델을 제공합니다
    backbone="resnet50",
    # 각 모델은 학습한 가중치를 가지고 있습니다
    # 이 가중치는 `keras_cv.model` 클래스의 가중치와 일치합니다.
    backbone_weights="imagenet",
    # include_rescaling는 모델에게 이미지의 픽셀값이 (0, 255)라는 것을 알려주거나
    # 이미 0과 255 사이의 값으로 조정했다면 그 값은 (0, 1) 사이라고 알려줍니다.
    # 예제의 경우, (0, 255)로 입력값을 사용합니다.
    include_rescaling=True,
    # 일반적으로 실제 모델을 학습할 때는 이 값은 False로 설정하려고 합니다.
    # evaluate_train_time_metrics=True 는 `train_step()`가 TPU와 호환되지 않습니다,
    # 또한, 많은 처리량이 필요합니다. 모델 학습 파이프라인을 디버깅할 때, 유용합니다
    evaluate_train_time_metrics=False,
)
# RetinaNet을 파인 튜닝하는 것은 backbone.trainable = False로 간단히 할 수 있습니다.
model.backbone.trainable = False